## Import Required Packages:

In [0]:
import os
import datetime
from collections import Counter
from pprint import pprint
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql import DataFrame as spark_DataFrame
from pyspark.sql.types import *
# from functools import reduce
import matplotlib.pyplot as plt
import requests
import re
from itertools import product
import time
from io import BytesIO
import requests
import numpy as np
from pyspark.sql import Window
import sys
#---------------------------- additional level2 packages ---------------------------------
from pyspark.sql.column import Column # _to_seq
from pyspark import copy_func, since
from pyspark.context import SparkContext
import genericpath
from genericpath import *
import stat
from pyspark.sql.column import _to_seq
import builtins
#------------------------------------- base import depencency ---------------------------
from __future__ import annotations
from pandas._libs import (NaT,Period,Timestamp,index as libindex,lib,)
from pandas.util._decorators import (
    Appender,
    Substitution,
    deprecate_kwarg,
    deprecate_nonkeyword_arguments,
    doc,
    rewrite_axis_style_signature,
)
from pandas._libs.lib import no_default

### Utility Functions

In [0]:
# base level data point to start 
_iord_zero_point = object()
# core engine algorithm for calculation and data arrangement 
def iord_algo(function, sequence, initial=_iord_zero_point):          
    """
    iord_algo(function, sequence[, initial]) -> value

    This is function of two arguments cumulatively of items of sequence, in right hand direction,
    so as to decrease sequence to single value. For example, iord_algo(lambda x, y: x+y, [1, 2, 3, 4, 5]) 
    process ((((1+2)+3)+4)+5).  Incase initial is avilable, it will be placed before the item
    of the sequence in the process, and serves as a default when for empty sequence.
    """

    iord_point = iter(sequence)

    if initial is _iord_zero_point:
        try:
            evaluate = next(iord_point)
        except StopIteration:
            raise TypeError("iord_algo() of empty sequence with no initial value") from None
    else:
        evaluate = initial
        
    for element in iord_point:
        evaluate = function(evaluate, element)

    return evaluate

In [0]:
#to give a lable to column
def _apply_grid_label_name(name):
    sc = SparkContext._active_spark_context
    return sc._jvm.functions.col(name)

if sys.version >= '3':
    basestring = str
    long = int

#to apply label with java jvm call
def _jvm_grid_label(col):
    if isinstance(col, Column):
        jcol = col._jc
    elif isinstance(col, basestring):
        jcol = _apply_grid_label_name(col)
    else:
        raise TypeError(
            "Invalid argument, should be either string or column: "
            "{0} of type {1}. "
            "For cols literals, please use lit, array, struct or create_map "
            "functions.".format(col, type(col)))
    return jcol

# convert date format into proper alignment
def date_alignment(date, format):
    sc = SparkContext._active_spark_context
    return Column(sc._jvm.functions.date_format(_jvm_grid_label(date), format))

# handle and validate begning date to pass jvm 
def beginning_date(col, format=None):
    sc = SparkContext._active_spark_context
    if format is None:
        jc = sc._jvm.functions.to_date(_jvm_grid_label(col))
    else:
        jc = sc._jvm.functions.to_date(_jvm_grid_label(col), format)
    return Column(jc)

# to re arrange columns for require format 
def intelora_rearrange_handler(col):
    sc = SparkContext._active_spark_context
    jc = sc._jvm.functions.explode(_jvm_grid_label(col))
    return Column(jc)

In [0]:
# align data with column and grid scale
def intelora_data_aligner(                             
        self,
        column: IndexLabel,
        ignore_index: bool = False,
    ) -> DataFrame:
        if not self.columns.is_unique:
            raise ValueError("columns is not unique, it has to be be unique-001")

        columns: list[Hashable]
        if is_scalar(column) or isinstance(column, tuple):
            columns = [column]
        elif isinstance(column, list) and all(
            map(lambda c: is_scalar(c) or isinstance(c, tuple), column)
        ):
            if not column:
                raise ValueError("column is empty, it should be nonempty")
            if len(column) > len(set(column)):
                raise ValueError("columns is not unique, it has to be be unique-002")
            columns = column
        else:
            raise ValueError("column is nut amoung any of scalar, tuple, or list thereof ")

        df = self.reset_index(drop=True)
        if len(columns) == 1:
            result = df[columns[0]].explode()
        else:
            mylen = lambda x: len(x) if is_list_like(x) else -1
            counts0 = self[columns[0]].apply(mylen)
            for c in columns[1:]:
                if not all(counts0 == self[c].apply(mylen)):
                    raise ValueError("columns should match element counts members")
            result = DataFrame({c: df[c].explode() for c in columns})
        result = df.drop(columns, axis=1).join(result)
        if ignore_index:
            result.index = default_index(len(result))
        else:
            result.index = self.index.take(result.index)
        result = result.reindex(columns=self.columns, copy=False)
        return result

# ------------------------ data structure for matching purpose -----------------------------------
def intelora_dataset_matcher(                                      
    obj: DataFrame,
    func: AggFuncType,
    axis: Axis = 0,
    raw: bool = False,
    result_type: str | None = None,
    args=None,
    kwargs=None,
) -> FrameApply:
    """It build and return a row or column based frame required object"""
    axis = obj._get_axis_number(axis)
    klass: type[FrameApply]
    if axis == 0:
        klass = FrameRowApply
    elif axis == 1:
        klass = FrameColumnApply

    return klass(
        obj,
        func,
        raw=raw,
        result_type=result_type,
        args=args,
        kwargs=kwargs,
    )
# ------------------------------------- clubbing data operation --------------------------------------------
def intelora_add(                                     
        self,
        func: AggFuncType,
        axis: Axis = 0,
        raw: bool = False,
        result_type=None,
        args=(),
        **kwargs,
    ):
        # the core appy of data frame matcher
        op = intelora_dataset_matcher(
            self,
            func=func,
            axis=axis,
            raw=raw,
            result_type=result_type,
            args=args,
            kwargs=kwargs,
        )
        return op.apply().__finalize__(self, method="apply")

In [0]:
# filter out data and argument with none and index label
@deprecate_nonkeyword_arguments(version=None, allowed_args=["self"])
def to_clean_require(
    self,
    axis: Axis = 0,
    how: str = "any",
    thresh=None,
    subset: IndexLabel = None,
    inplace: bool = False,
    ):
        inplace = validate_bool_kwarg(inplace, "inplace")
        if isinstance(axis, (tuple, list)):
            raise TypeError(" it not support anymore supplying multiple axes to axis.")
        axis = self._get_axis_number(axis)
        agg_axis = 1 - axis

        agg_obj = self
        if subset is not None:
            # subset require list format only
            if not is_list_like(subset):
                subset = [subset]
            ax = self._get_axis(agg_axis)
            indices = ax.get_indexer_for(subset)
            check = indices == -1
            if check.any():
                raise KeyError(np.array(subset)[check].tolist())
            agg_obj = self.take(indices, axis=agg_axis)

        if thresh is not None:
            count = agg_obj.count(axis=agg_axis)
            mask = count >= thresh
        elif how == "any":
            # faster compare to 'agg_obj.count(agg_axis) == self.shape[agg_axis]'
            mask = notna(agg_obj).all(axis=agg_axis, bool_only=False)
        elif how == "all":
            # faster compare to 'agg_obj.count(agg_axis) > 0'
            mask = notna(agg_obj).any(axis=agg_axis, bool_only=False)
        else:
            if how is not None:
                raise ValueError(f"not valid how option: {how}")
            else:
                raise TypeError("should specify how or thresh")

        if np.all(mask):
            result = self.copy()
        else:
            result = self.loc(axis=axis)[mask]

        if inplace:
            self._update_inplace(result)
        else:
            return result

#------------------------------------- data structure collpse ------------------------------------------------------
def intelora_arranger(           
    self,
    by=None,
    axis: Axis = 0,
    level: Level | None = None,
    as_index: bool = True,
    sort: bool = True,
    group_keys: bool = True,
    squeeze: bool | lib.NoDefault = no_default,
    observed: bool = False,
    dropna: bool = True,
) -> DataFrameGroupBy:
    from pandas.core.groupby.generic import DataFrameGroupBy
    # validating data in sequence
    if squeeze is not no_default:
        warnings.warn(
            (
               " squeeze parameter is no longer supported and "
                "suppose to removed later."
            ),
            FutureWarning,
            stacklevel=find_stack_level(),
        )
    else:
        squeeze = False

    if level is None and by is None:
        raise TypeError("You have to supply one of 'by' and 'level'")
    axis = self._get_axis_number(axis)
    return DataFrameGroupBy(
        obj=self,
        keys=by,
        axis=axis,
        level=level,
        as_index=as_index,
        sort=sort,
        group_keys=group_keys,
        squeeze=squeeze,  # type: ignore[arg-type]
        observed=observed,
        dropna=dropna,
    )

In [0]:
# apply call leavel data predefined rule 
def also_call_as(c):
    """
    create list of aliases for given company using predefined rules.
    Try to encompass as many options as possible keeping so that user can customize,
    containg aliases may left out.
    """
    c = c.lower()

    # maintain URLs string such as .com
    if len(c) > 3 and c[-4] == ".":
        a = c
        c = c.rsplit(".", 1)[0].replace(".", " ")
        aliases = set([c, a])
    else:
        aliases = set([c])

    # get last character of endings
    if len(c.split()[-1]) == 1:
        c = c.rsplit(" ", 1)[0]
        aliases.add(c)

    # Company legal endings
    endings = ["inc",
               "corp",
               "plc", 
               "reit", 
               "co", 
               "cor", 
               "group", 
               "company",
               "trust",
               "energy",
               "international",
               "of america",
               "pharmaceuticals",
               "clas",
               "in", "nv",
               "sa", 
               "re", 
               "pvt ltd",
               "private limited" ,
               "india private limited"
               "Co",
               "CO.",
               "Companies",
               "Company",
               "Corp",
               "CORP.",
               "Corporation",
               "Inc",
               "INC.",
               "Incorporated",
               "Limited",
               "Ltd",
               "Professional Corporation",
               "Chartered",
               "Limited",
               "Ltd",
               "Ltd.",
               "pa",
               "p.c ",
               "Professional Association",
               "Professional Corporation",
               "Professional Service Corporation",
               "psc",
               "sc.",
               "Service Corporation"]
    
    n_endings = 3  # Can have up to 3 of these endings
    for _ in range(n_endings):
        aliases.update([a.rsplit(" ", 1)[0] for a in aliases if
                        any([a.endswith(" " + e) for e in endings])])
        c = c.rsplit(" ", 1)[0] if any([c.endswith(" " + e) for e in endings]) else c

    # Alias any dashes and replace in company name
    aliases.update([a.replace("-", "") for a in aliases] +
                   [a.replace("-", " ") for a in aliases])
    c = c.replace("-", " ")

    # If '&' stands on its own, add alias of 'and'
    aliases.update([a.replace(" & ", " and ") for a in aliases])
    return {c: list(aliases)}

# get column alias from compnay 
def generate_common_org_names(companies):
    """
    store data of  companies and loop through them to find aliases
    """
    #companies = [org_name]
    comp_dict = dict()
    for c in companies:
        comp_dict.update(also_call_as(c))
    return comp_dict

#using spark dataframe union for list
combine_spark_dfs = lambda sdf_list: iord_algo(spark_DataFrame.union, sdf_list)

# a global level common data defination
class Common_MetaData:
    """ Variables to use across many functions. """
    keep = ["DATE",
            "SourceCommonName", 
            "DocumentIdentifier", 
            "Themes",
            "Organizations",
            "V2Tone"]
    tone = ["Tone",
            "PositiveTone",
            "NegativeTone", 
            "Polarity",
            "ActivityDensity", 
            "SelfDensity",
            "WordCount"]
    organizations = None

# spark based expand basic dataset
@udf(ArrayType(StringType(), True))
def simple_expand_spark(x):
    """ Expand semicolon token strint to a list (also avoid empty)"""
    if not x:
        return []
    return list(filter(None, x.split(";")))

# spark based tone data generation
@udf(MapType(StringType(), DoubleType()))
def tone_expand_spark(x):
    """ Expand and identify tone field. """
    if not x:
        return {t: None for t in Common_MetaData.tone}
    return {Common_MetaData.tone[i]: float(v) for i, v in enumerate(x.split(","))}

# spark template validation for stream
@udf(BooleanType())
def has_theme_spark(x, theme):
    """ Is the given theme included in any of the listed themes? """
    return any([theme in lst.split("_") for lst in x])

# cleanup process for org data 
@udf(StringType())
def clean_organization(s):
    """ Standardize the organization names. """
    for k, v in Common_MetaData.organizations.items():
        if v[0] in s.split() :
            return k
    return s.lower()

## alias of company and compass data value
# def also_call_create(c):
#     """
#     create list of aliases for require company using few predefined rules.
#     Try to encompass many options with possible keeping possibality,
#     keep needed aliases left out.
#     """
#     c = c.lower()

#     # URLs (e.g. ".com")
#     if len(c) > 3 and c[-4] == ".":
#         a = c
#         c = c.rsplit(".", 1)[0].replace(".", " ")
#         aliases = set([c, a])
#     else:
#         aliases = set([c])
#     return aliases

## Downloading , Filtering and Storing Gdelt Data:

In [0]:
# Data structure format to help to arrange data in various format
class Redesign_data_format:
    # spark code to redesign data frame based on matadata  
    def redesign_sdf(self, sdf, file_path_refined, org_name):
        """
        Provide spark data frame of the downloaded data and then reformat
        it into human-readable all Common_MetaData.
        Also it add few more Common_MetaData for structuring purposes.
        """
        setattr(Common_MetaData, "organizations",generate_common_org_names(org_name))
        
        sdf = sdf.select(*Common_MetaData.keep)
        
        if not os.path.exists(file_path_refined.replace("dbfs:/", "/dbfs/")):
            # manage existing columns
            sdf = (sdf.withColumnRenamed("DocumentIdentifier", "URL")
                      .withColumn("Themes", simple_expand_spark("Themes"))
                      .withColumn("Organizations", simple_expand_spark("Organizations"))
                      .withColumn("V2Tone", tone_expand_spark("V2Tone"))
                   )

            # Create ESG columns & explode organization column
            sdf = (sdf.withColumn("E", has_theme_spark("Themes", F.lit("ENV")))
                      .withColumn("S", has_theme_spark("Themes", F.lit("UNGP")))
                      .withColumn("G", has_theme_spark("Themes", F.lit("ECON")))                      
                   )
        
            sdf.write.format("delta").option("header", "true").mode("overwrite").save(file_path_refined)
            print("sdf created")
        
        else:
            sdf = spark.read.format("delta").option("header", "true").load(file_path_refined)    
        
        sdf = (sdf.withColumn("Organization", intelora_rearrange_handler("Organizations"))
                  .withColumn("Organization", clean_organization("organization"))
                  .filter(F.col("organization").isin(list(Common_MetaData.organizations.keys())))
              )

        # Expand tone columns
        exprs = [F.col("V2Tone").getItem(k).alias(k) for k in Common_MetaData.tone]

        sdf = sdf.select(*sdf.columns, *exprs).drop("V2Tone")
        #print(sdf.column)
        return sdf

## download gdelt dat and generate table
#     def download_and_generate_gdelt_table1(self, date, gd):
#         """
#         Download the GDELT table as a pandas dataframe using the gdelt package.
#         Return a spark data frame.
#         """
#         pdf = gd.Search([date], table="gkg",coverage=True, output="df")
#         pdf["DATE"] = pd.to_datetime(pdf["DATE"], format="%Y%m%d%H%M%S")

#         sdf = spark.createDataFrame(pdf)
#         print("   * loaded *  ", date)
#         return sdf
    # download gdelt dat and generate table helper function
    def download_and_generate_gdelt_table(self, date, file_path):
        file_path_os =  file_path.replace("dbfs:/", "/dbfs/")
        s = []
        for i in range(24):
            for j in list(range(0, 60, 15)):
                if i<10:
                    if j<10:
                        s.append('0' + str(i) + '0' + str(j))
                    else:
                        s.append('0' + str(i) + str(j))
                else:
                    if j<10:
                        s.append(str(i) + '0' + str(j))
                    else:
                         s.append(str(i) + str(j))
        
        #validing file path URL
        if not os.path.exists(file_path_os):
            li = ''.join(date.split('-'))
            c=0
            df1 = None
            for elem in s:
                try:
                    print(li, elem)
                    response = requests.get('http://data.gdeltproject.org/gdeltv2/'+li+ elem + '00.gkg.csv.zip')
                    #for example actual url is = 'http://data.gdeltproject.org/gkg/20220204.gkg.csv.zip')
                    buffer = BytesIO(response.content)
                   
                    frame = pd.read_csv(buffer, compression='zip', sep='\t',header=None, warn_bad_lines=True,encoding='latin')
                    frame[1] = pd.to_datetime(frame[1], format="%Y%m%d%H%M%S")
        
                    frame.columns = ['GKGRECORDID', 
                                     'DATE', 
                                     'SourceCollectionIdentifier', 
                                     'SourceCommonName',
                                     'DocumentIdentifier',
                                     'Counts',
                                     'V2Counts', 
                                     'Themes', 
                                     'V2Themes',
                                     'Locations',
                                     'V2Locations', 
                                     'Persons', 
                                     'V2Persons', 
                                     'Organizations',
                                     'V2Organizations', 
                                     'V2Tone',
                                     'Dates',
                                     'GCAM',
                                     'SharingImage',
                                     'RelatedImages', 
                                     'SocialImageEmbeds', 
                                     'SocialVideoEmbeds', 
                                     'Quotations',
                                     'AllNames', 
                                     'Amounts', 
                                     'TranslationInfo', 
                                     'Extras'] 
                    columns1 = ['DATE', 
                                'SourceCollectionIdentifier', 
                                'SourceCommonName',
                                'DocumentIdentifier',
                                'Counts', 
                                'V2Counts',
                                'Themes',
                                'V2Themes',
                                'Locations',
                                'V2Locations',
                                'Organizations',
                                'V2Organizations',
                                'V2Tone',
                                'Dates'] 
                    frame = frame[columns1]
                    print(frame.shape)
                    if c==0:
                        df1 = frame
                        c=1
                    else:
                        df1 = df1.append(frame, ignore_index=True)
                except:
                    pass
            
            sdf = spark.createDataFrame(df1)
            sdf.write.format("delta").option("header", "true").mode("overwrite").save(file_path)
            print("   * loaded *  ", date)
        else:
            sdf = spark.read.format("delta").option("header", "true").load(file_path)
        return sdf

    # function to get all data for input organization
    def getting_all_org_data(self, start_date, end_date, main_dir , refined_dir,organisation_name = None, override_save=False):
        """
        For each date between start_date and end_date, either download
        and clean the data or load the pre-saved data. Save the day's data
        in case of future use (so it doesn't have to be downloaded and cleaned again)
        """
        print("Loading and cleaning all data")
        data_list = []
        
        # Download and format the daily data
        for i, date in enumerate(pd.date_range(start_date, end_date).astype(str)):
            if i % 7 == 1:
                # Prevent it hanging like it does sometimes
                time.sleep(60)

            try:
                #file_path = os.path.join(main_dir, date)
                file_path = os.path.join(main_dir, date)
                file_path_refined = os.path.join(refined_dir, date)
                df = self.redesign_sdf(self.download_and_generate_gdelt_table(date, file_path), file_path_refined, organisation_name)      
                data_list.append(df)
                del df
                spark.catalog.clearCache()

            except Exception as e:
              print(f"!!! Failed to complete {date}!")
              print("  ****   Reason:\n" + str(e) + "\n\n")
        return combine_spark_dfs(data_list)

## function to download and formate data into human readable format
#     def reformat_sdf(self, sdf, file_path_refined, org_name):
#         """
#         Given a spark data frame of the downloaded data, reformat it
#         into human-readable Common_MetaData.
#         Add a few more Common_MetaData for our purposes.
#         """
#         sdf = sdf.select(*Common_MetaData.keep)
        
#         if not os.path.exists(file_path_refined.replace("dbfs:/", "/dbfs/")):
#             # Reformat existing columns
#             sdf = (sdf.withColumnRenamed("DocumentIdentifier", "URL")
#                       .withColumn("Themes", simple_expand_spark("Themes"))
#                       .withColumn("Organizations", simple_expand_spark("Organizations"))
#                       .withColumn("V2Tone", tone_expand_spark("V2Tone"))
#                    )

#             # Create ESG columns & explode organization column
#             sdf = (sdf.withColumn("E", has_theme_spark("Themes", F.lit("ENV")))
#                       .withColumn("S", has_theme_spark("Themes", F.lit("UNGP")))
#                       .withColumn("G", has_theme_spark("Themes", F.lit("ECON")))                      
#                    )
        
#             sdf.write.format("delta").option("header", "true").mode("overwrite").save(file_path_refined)
#             print("sdf created")
        
#         else:
#             sdf = spark.read.format("delta").option("header", "true").load(file_path_refined)    
        
#         sdf = (sdf.withColumn("Organization", F.explode("Organizations"))
#                   .withColumn("Organization", clean_organization("organization"))
#                   .filter(F.col("organization").isin(list(Common_MetaData.organizations.keys())))
#               )

#         return sdf

# gdelt data pre process to filter out for process before start
#      def preprocess_gdelt_data(self ,start_date, end_date, org_name = None, save_csv=True):
#         """
#         """   
#         dbutils.fs.rm('/mnt/esg/financial_report_data/GDELT_data_single_org/data_single_org', True)    

#         base_dir = f"dbfs:/mnt/esg/financial_report_data/GDELT_data_single_org"
#         if not os.path.exists(base_dir.replace("dbfs:/", "/dbfs/")):
#             dbutils.fs.mkdirs(base_dir)

#         org_dir = 'dbfs:/mnt/esg/financial_report_data/GDELT_data_single_org/data_single_org'
#         if not os.path.exists(org_dir.replace("dbfs:/", "/dbfs/")):
#             dbutils.fs.mkdirs(org_dir)

#         base_data_dir = f"dbfs:/mnt/esg/financial_report_data/GDELT_data_single_org/data"
#         base_data_dir_refined = f"dbfs:/mnt/esg/financial_report_data/GDELT_data_single_org/data_refined"

#         if not os.path.exists(base_data_dir.replace("dbfs:/", "/dbfs/")):
#             dbutils.fs.mkdirs(base_data_dir)
#         if not os.path.exists(base_data_dir_refined.replace("dbfs:/", "/dbfs/")):     
#             dbutils.fs.mkdirs(base_data_dir_refined)
#         # Download and reformat the data
#         print('')
#         data = Redesign_data_format().getting_all_org_data(start_date, end_date, base_data_dir, base_data_dir_refined, org_name)
#         print(f"There are {data.count():,d} data points for {len(Common_MetaData.organizations)} "
#               f"organizations from {start_date} to {end_date}")

#         # Save the data
#         print("Saving Data...")
#         data_save_path = f"dbfs:/mnt/esg/financial_report_data/GDELT_data_single_org/data_single_org" ## we are changing
#         if not os.path.exists(base_data_dir.replace("dbfs:/", "/dbfs/")):
#             dbutils.fs.mkdirs(base_data_dir)
#         data.write.format("delta").mode("overwrite").save(data_save_path) ## need to remove this..
#         print(f"Saved to {data_save_path}")

#         return data
#     # COMMAND ----------

# connector function to connect with gddelt and pull data
def download_gdelt_data(start_date, end_date, org_name = None, save_csv=True):
    """
    """   
    dbutils.fs.rm('/mnt/esg/financial_report_data/GDELT_data_single_org/data_single_org', True)    

    base_dir = f"dbfs:/mnt/esg/financial_report_data/GDELT_data_single_org"
    if not os.path.exists(base_dir.replace("dbfs:/", "/dbfs/")):
        dbutils.fs.mkdirs(base_dir)

    org_dir = 'dbfs:/mnt/esg/financial_report_data/GDELT_data_single_org/data_single_org'
    if not os.path.exists(org_dir.replace("dbfs:/", "/dbfs/")):
        dbutils.fs.mkdirs(org_dir)

    base_data_dir = f"dbfs:/mnt/esg/financial_report_data/GDELT_data_single_org/data"
    base_data_dir_refined = f"dbfs:/mnt/esg/financial_report_data/GDELT_data_single_org/data_refined"

    if not os.path.exists(base_data_dir.replace("dbfs:/", "/dbfs/")):
        dbutils.fs.mkdirs(base_data_dir)
    if not os.path.exists(base_data_dir_refined.replace("dbfs:/", "/dbfs/")):     
        dbutils.fs.mkdirs(base_data_dir_refined)
        
    # Download and reformat the data
    print('')
    data = Redesign_data_format().getting_all_org_data(start_date, end_date, base_data_dir, base_data_dir_refined, org_name)
    print(f"There are {data.count():,d} data points for {len(Common_MetaData.organizations)} "
          f"organizations from {start_date} to {end_date}")

    # Save the data
    print("Saving Data...")
    data_save_path = f"dbfs:/mnt/esg/financial_report_data/GDELT_data_single_org/data_single_org" ## we are changing
    if not os.path.exists(base_data_dir.replace("dbfs:/", "/dbfs/")):
        dbutils.fs.mkdirs(base_data_dir)
    data.write.format("delta").mode("overwrite").save(data_save_path) ## need to remove this..
    print(f"Saved-001 to {data_save_path}")

    return data


## ESG Computating Functions:

In [0]:
from __future__ import annotations
from pandas.core.dtypes.generic import (
    ABCDataFrame,
    ABCSeries,
)

Global_identifierlabel = """
        Parameters
        ----------
        buf : str, Path or StringIO-like, optional, default None
            Buffer to write to. If None, the output is returned as a string.
        columns : sequence, optional, default None
            The subset of columns to write. Writes all columns by default.
        col_space : %(col_space_type)s, optional
            %(col_space)s.
        header : %(header_type)s, optional
            %(header)s.
        index : bool, optional, default True
            Whether to print index (row) labels.
        na_rep : str, optional, default 'NaN'
            String representation of ``NaN`` to use.
        formatters : list, tuple or dict of one-param. functions, optional
            Formatter functions to apply to columns' elements by position or
            name.
            The result of each function must be a unicode string.
            List/tuple must be of length equal to the number of columns.
        float_format : one-parameter function, optional, default None
            Formatter function to apply to columns' elements if they are
            floats. This function must return a unicode string and will be
            applied only to the non-``NaN`` elements, with ``NaN`` being
            handled by ``na_rep``.

            .. versionchanged:: 1.2.0

        sparsify : bool, optional, default True
            Set to False for a DataFrame with a hierarchical index to print
            every multiindex key at each row.
        index_names : bool, optional, default True
            Prints the names of the indexes.
        justify : str, default None
            How to justify the column labels. If None uses the option from
            the print configuration (controlled by set_option), 'right' out
            of the box. Valid values are

            * left
            * right
            * center
            * justify
            * justify-all
            * start
            * end
            * inherit
            * match-parent
            * initial
            * unset.
        max_rows : int, optional
            Maximum number of rows to display in the console.
        max_cols : int, optional
            Maximum number of columns to display in the console.
        show_dimensions : bool, default False
            Display DataFrame dimensions (number of rows by number of columns).
        decimal : str, default '.'
            Character recognized as decimal separator, e.g. ',' in Europe.
    """
Give_identifierlabel = """
        Returns
        -------
        str or None
            If buf is None, returns the result as a string. Otherwise returns
            None.
    """
class InteloraRecordHandler:
    """Class for processing dataframe formatting options and data."""

    __doc__ = __doc__ if __doc__ else ""
    __doc__ += Global_identifierlabel + Give_identifierlabel

    def __init__(
        self,
        frame: DataFrame,
        columns: Sequence[str] | None = None,
        col_space: ColspaceArgType | None = None,
        header: bool | Sequence[str] = True,
        index: bool = True,
        na_rep: str = "NaN",
        formatters: FormattersType | None = None,
        justify: str | None = None,
        float_format: FloatFormatType | None = None,
        sparsify: bool | None = None,
        index_names: bool = True,
        max_rows: int | None = None,
        min_rows: int | None = None,
        max_cols: int | None = None,
        show_dimensions: bool | str = False,
        decimal: str = ".",
        bold_rows: bool = False,
        escape: bool = True,
    ):
        self.frame = frame
        self.columns = self._initl_cols(columns)
        self.col_space = self._initl_colsp(col_space)
        self.header = header
        self.index = index
        self.na_rep = na_rep
        self.formatters = self._initl_metadata_type(formatters)
        self.justify = self._initl_caliber(justify)
        self.float_format = float_format
        self.sparsify = self._initl_distribute(sparsify)
        self.show_index_names = index_names
        self.decimal = decimal
        self.bold_rows = bold_rows
        self.escape = escape
        self.max_rows = max_rows
        self.min_rows = min_rows
        self.max_cols = max_cols
        self.show_dimensions = show_dimensions

        self.max_cols_fitted = self._get_cols_max_size()
        self.max_rows_fitted = self._get_rows_max_size()

        self.tr_frame = self.frame
        self.intelora_triming_data()
        self.adj = get_adjustment()

    def intelora_char_getcols(self) -> list[list[str]]:
        """
        Render a DataFrame to a list of columns (as lists of strings).
        """
        strcols = self._get_index_un_strcols()

        if self.index:
            str_index = self._get_index_metadata_type(self.tr_frame)
            strcols.insert(0, str_index)

        return strcols

    @property
    def matrix_to_display(self) -> bool:
        return self.show_dimensions is True or (
            self.show_dimensions == "truncate" and self.check_trimed
        )

    @property
    def check_trimed(self) -> bool:
        return bool(self.check_htrimed or self.check_vtrimed)

    @property
    def check_htrimed(self) -> bool:
        return bool(self.max_cols_fitted and (len(self.columns) > self.max_cols_fitted))

    @property
    def check_vtrimed(self) -> bool:
        return bool(self.max_rows_fitted and (len(self.frame) > self.max_rows_fitted))

    @property
    def intelora_info_matrix(self) -> str:
        return f"\n\n[{len(self.frame)} rows x {len(self.frame.columns)} columns]"

    @property
    def find_named_index(self) -> bool:
        return _has_names(self.frame.index)

    @property
    def find_named_columns(self) -> bool:
        return _has_names(self.frame.columns)

    @property
    def display_row_sequence_name(self) -> bool:
        return all((self.find_named_index, self.index, self.show_index_names))

    @property
    def display_col_sequence_name(self) -> bool:
        return all((self.find_named_columns, self.show_index_names, self.header))

    @property
    def frame_all_rows(self) -> int:
        return min(self.max_rows or len(self.frame), len(self.frame))

    def _initl_distribute(self, sparsify: bool | None) -> bool:
        if sparsify is None:
            return get_option("display.multi_sparse")
        return sparsify

    def _initl_metadata_type(
        self, formatters: FormattersType | None
    ) -> FormattersType:
        if formatters is None:
            return {}
        elif len(self.frame.columns) == len(formatters) or isinstance(formatters, dict):
            return formatters
        else:
            raise ValueError(
                f"Formatters length({len(formatters)}) should match "
                f"DataFrame number of columns({len(self.frame.columns)})"
            )

    def _initl_caliber(self, justify: str | None) -> str:
        if justify is None:
            return get_option("display.colheader_justify")
        else:
            return justify

    def _initl_cols(self, columns: Sequence[str] | None) -> Index:
        if columns is not None:
            cols = ensure_index(columns)
            self.frame = self.frame[cols]
            return cols
        else:
            return self.frame.columns

    def _initl_colsp(self, col_space: ColspaceArgType | None) -> ColspaceType:
        result: ColspaceType

        if col_space is None:
            result = {}
        elif isinstance(col_space, (int, str)):
            result = {"": col_space}
            result.update({column: col_space for column in self.frame.columns})
        elif isinstance(col_space, Mapping):
            for column in col_space.keys():
                if column not in self.frame.columns and column != "":
                    raise ValueError(
                        f"Col_space is defined for an unknown column: {column}"
                    )
            result = col_space
        else:
            if len(self.frame.columns) != len(col_space):
                raise ValueError(
                    f"Col_space length({len(col_space)}) should match "
                    f"DataFrame number of columns({len(self.frame.columns)})"
                )
            result = dict(zip(self.frame.columns, col_space))
        return result

    def _get_cols_max_size(self) -> int | None:
        """Number of columns fitting the screen."""
        if not self._intelora_check_command_validator():
            return self.max_cols

        width, _ = get_terminal_size()
        if self._check_display_frame(width):
            return width
        else:
            return self.max_cols

    def _get_rows_max_size(self) -> int | None:
        """Number of rows with data fitting the screen."""
        max_rows: int | None

        if self._intelora_check_command_validator():
            _, height = get_terminal_size()
            if self.max_rows == 0:
                # rows available to fill with actual data
                return height - self._get_row_count_auxillary()

            if self._check_display_small(height):
                max_rows = height
            else:
                max_rows = self.max_rows
        else:
            max_rows = self.max_rows

        return self._mix_value_alignment(max_rows)

    def _mix_value_alignment(self, max_rows: int | None) -> int | None:
        """Adjust max_rows using display logic.

        See description here:
        https://pandas.pydata.org/docs/dev/user_guide/options.html#frequently-used-options

        GH #37359
        """
        if max_rows:
            if (len(self.frame) > max_rows) and self.min_rows:
                # if truncated, set max_rows showed to min_rows
                max_rows = min(self.min_rows, max_rows)
        return max_rows

    def _intelora_check_command_validator(self) -> bool:
        """Check if the output is to be shown in terminal."""
        return bool(self.max_cols == 0 or self.max_rows == 0)

    def _check_display_frame(self, max_width) -> bool:
        return bool(self.max_cols == 0 and len(self.frame.columns) > max_width)

    def _check_display_small(self, max_height) -> bool:
        return bool(self.max_rows == 0 and len(self.frame) > max_height)

    def _get_row_count_auxillary(self) -> int:
        """Get number of rows occupied by prompt, dots and dimension info."""
        dot_row = 1
        prompt_row = 1
        num_rows = dot_row + prompt_row

        if self.show_dimensions:
            num_rows += len(self.intelora_info_matrix.splitlines())

        if self.header:
            num_rows += 1

        return num_rows

    def intelora_triming_data(self) -> None:
        """
        Check whether the frame should be truncated. If so, slice the frame up.
        """
        if self.check_trimed_horizontally:
            self._direct_htrim()

        if self.check_trimed_vertically:
            self._direct_vtrim()

    def _direct_htrim(self) -> None:
        """Remove columns, which are not to be displayed and adjust formatters.

        Attributes affected:
            - tr_frame
            - formatters
            - tr_col_num
        """
        assert self.max_cols_fitted is not None
        col_num = self.max_cols_fitted // 2
        if col_num >= 1:
            left = self.tr_frame.iloc[:, :col_num]
            right = self.tr_frame.iloc[:, -col_num:]
            self.tr_frame = concat((left, right), axis=1)

            # truncate formatter
            if isinstance(self.formatters, (list, tuple)):
                self.formatters = [
                    *self.formatters[:col_num],
                    *self.formatters[-col_num:],
                ]
        else:
            col_num = cast(int, self.max_cols)
            self.tr_frame = self.tr_frame.iloc[:, :col_num]
        self.tr_col_num = col_num

    def _direct_vtrim(self) -> None:
        """Remove rows, which are not to be displayed.

        Attributes affected:
            - tr_frame
            - tr_row_num
        """
        assert self.max_rows_fitted is not None
        row_num = self.max_rows_fitted // 2
        if row_num >= 1:
            head = self.tr_frame.iloc[:row_num, :]
            tail = self.tr_frame.iloc[-row_num:, :]
            self.tr_frame = concat((head, tail))
        else:
            row_num = cast(int, self.max_rows)
            self.tr_frame = self.tr_frame.iloc[:row_num, :]
        self.tr_row_num = row_num

    def _get_index_un_strcols(self) -> list[list[str]]:
        strcols: list[list[str]] = []

        if not is_list_like(self.header) and not self.header:
            for i, c in enumerate(self.tr_frame):
                fmt_values = self.collumn_metadata(i)
                fmt_values = _make_fixed_width(
                    strings=fmt_values,
                    justify=self.justify,
                    minimum=int(self.col_space.get(c, 0)),
                    adj=self.adj,
                )
                strcols.append(fmt_values)
            return strcols

        if is_list_like(self.header):
            # cast here since can't be bool if is_list_like
            self.header = cast(List[str], self.header)
            if len(self.header) != len(self.columns):
                raise ValueError(
                    f"Writing {len(self.columns)} cols "
                    f"but got {len(self.header)} aliases"
                )
            str_columns = [[label] for label in self.header]
        else:
            str_columns = self._get_column_labels_metadata_type(self.tr_frame)

        if self.display_row_sequence_name:
            for x in str_columns:
                x.append("")

        for i, c in enumerate(self.tr_frame):
            cheader = str_columns[i]
            header_colwidth = max(
                int(self.col_space.get(c, 0)), *(self.adj.len(x) for x in cheader)
            )
            fmt_values = self.collumn_metadata(i)
            fmt_values = _make_fixed_width(
                fmt_values, self.justify, minimum=header_colwidth, adj=self.adj
            )

            max_len = max(max(self.adj.len(x) for x in fmt_values), header_colwidth)
            cheader = self.adj.justify(cheader, max_len, mode=self.justify)
            strcols.append(cheader + fmt_values)

        return strcols

    def collumn_metadata(self, i: int) -> list[str]:
        frame = self.tr_frame
        formatter = self._get_metadata_type(i)
        return format_array(
            frame.iloc[:, i]._values,
            formatter,
            float_format=self.float_format,
            na_rep=self.na_rep,
            space=self.col_space.get(frame.columns[i]),
            decimal=self.decimal,
            leading_space=self.index,
        )

    def _get_metadata_type(self, i: str | int) -> Callable | None:
        if isinstance(self.formatters, (list, tuple)):
            if is_integer(i):
                i = cast(int, i)
                return self.formatters[i]
            else:
                return None
        else:
            if is_integer(i) and i not in self.columns:
                i = self.columns[i]
            return self.formatters.get(i, None)

    def _get_column_labels_metadata_type(self, frame: DataFrame) -> list[list[str]]:
        from pandas.core.indexes.multi import sparsify_labels

        columns = frame.columns

        if isinstance(columns, MultiIndex):
            fmt_columns = columns.format(sparsify=False, adjoin=False)
            fmt_columns = list(zip(*fmt_columns))
            dtypes = self.frame.dtypes._values

            # if we have a Float level, they don't use leading space at all
            restrict_formatting = any(level.is_floating for level in columns.levels)
            need_leadsp = dict(zip(fmt_columns, map(is_numeric_dtype, dtypes)))

            def void_arrangement(x, y):
                if (
                    y not in self.formatters
                    and need_leadsp[x]
                    and not restrict_formatting
                ):
                    return " " + y
                return y

            str_columns = list(
                zip(*([void_arrangement(x, y) for y in x] for x in fmt_columns))
            )
            if self.sparsify and len(str_columns):
                str_columns = sparsify_labels(str_columns)

            str_columns = [list(x) for x in zip(*str_columns)]
        else:
            fmt_columns = columns.format()
            dtypes = self.frame.dtypes
            need_leadsp = dict(zip(fmt_columns, map(is_numeric_dtype, dtypes)))
            str_columns = [
                [" " + x if not self._get_metadata_type(i) and need_leadsp[x] else x]
                for i, x in enumerate(fmt_columns)
            ]
        # self.str_columns = str_columns
        return str_columns

    def _get_index_metadata_type(self, frame: DataFrame) -> list[str]:
        # Note: this is only used by to_string() and to_latex(), not by
        # to_html(). so safe to cast col_space here.
        col_space = {k: cast(int, v) for k, v in self.col_space.items()}
        index = frame.index
        columns = frame.columns
        fmt = self._get_metadata_type("__index__")

        if isinstance(index, MultiIndex):
            fmt_index = index.format(
                sparsify=self.sparsify,
                adjoin=False,
                names=self.display_row_sequence_name,
                formatter=fmt,
            )
        else:
            fmt_index = [index.format(name=self.display_row_sequence_name, formatter=fmt)]

        fmt_index = [
            tuple(
                _make_fixed_width(
                    list(x), justify="left", minimum=col_space.get("", 0), adj=self.adj
                )
            )
            for x in fmt_index
        ]

        adjoined = self.adj.adjoin(1, *fmt_index).split("\n")

        # empty space for columns
        if self.display_col_sequence_name:
            col_header = [str(x) for x in self._get_column_names()]
        else:
            col_header = [""] * columns.nlevels

        if self.header:
            return col_header + adjoined
        else:
            return adjoined

    def _get_column_names(self) -> list[str]:
        names: list[str] = []
        columns = self.frame.columns
        if isinstance(columns, MultiIndex):
            names.extend("" if name is None else name for name in columns.names)
        else:
            names.append("" if columns.name is None else columns.name)
        return names
#--------------------------------------------------------------------------------------------------------------------------------------------
class InteloraRecordGenerator:

    def __init__(self, fmt: InteloraRecordHandler):
        self.fmt = fmt

    def iord_clay_format(
        self,
        buf: FilePath | WriteBuffer[str] | None = None,
        column_format: str | None = None,
        longtable: bool = False,
        encoding: str | None = None,
        multicolumn: bool = False,
        multicolumn_format: str | None = None,
        multirow: bool = False,
        caption: str | None = None,
        label: str | None = None,
        position: str | None = None,
    ) -> str | None:
        """
        Render a DataFrame to a LaTeX tabular/longtable environment output.
        """
        from pandas.io.formats.latex import LatexFormatter

        latex_formatter = LatexFormatter(
            self.fmt,
            longtable=longtable,
            column_format=column_format,
            multicolumn=multicolumn,
            multicolumn_format=multicolumn_format,
            multirow=multirow,
            caption=caption,
            label=label,
            position=position,
        )
        string = latex_formatter.to_string()
        return save_to_buffer(string, buf=buf, encoding=encoding)

    def iord_web_format(
        self,
        buf: FilePath | WriteBuffer[str] | None = None,
        encoding: str | None = None,
        classes: str | list | tuple | None = None,
        notebook: bool = False,
        border: int | None = None,
        table_id: str | None = None,
        render_links: bool = False,
    ) -> str | None:
        
        from pandas.io.formats.html import (
            HTMLFormatter,
            NotebookFormatter,
        )

        Klass = NotebookFormatter if notebook else HTMLFormatter

        html_formatter = Klass(
            self.fmt,
            classes=classes,
            border=border,
            table_id=table_id,
            render_links=render_links,
        )
        string = html_formatter.to_string()
        return save_to_buffer(string, buf=buf, encoding=encoding)

    def string_formlizer(
        self,
        buf: FilePath | WriteBuffer[str] | None = None,
        encoding: str | None = None,
        line_width: int | None = None,
    ) -> str | None:
        
        from pandas.io.formats.string import StringFormatter

        string_formatter = StringFormatter(self.fmt, line_width=line_width)
        string = string_formatter.to_string()
        return save_to_buffer(string, buf=buf, encoding=encoding)

    def csv_generator(
        self,
        path_or_buf: FilePath | WriteBuffer[bytes] | WriteBuffer[str] | None = None,
        encoding: str | None = None,
        sep: str = ",",
        columns: Sequence[Hashable] | None = None,
        index_label: IndexLabel | None = None,
        mode: str = "w",
        compression: CompressionOptions = "infer",
        quoting: int | None = None,
        quotechar: str = '"',
        line_terminator: str | None = None,
        chunksize: int | None = None,
        date_format: str | None = None,
        doublequote: bool = True,
        escapechar: str | None = None,
        errors: str = "strict",
        storage_options: StorageOptions = None,
    ) -> str | None:
        """
        Render dataframe as comma-separated file.
        """
        from pandas.io.formats.csvs import CSVFormatter

        if path_or_buf is None:
            created_buffer = True
            path_or_buf = StringIO()
        else:
            created_buffer = False

        csv_formatter = CSVFormatter(
            path_or_buf=path_or_buf,
            line_terminator=line_terminator,
            sep=sep,
            encoding=encoding,
            errors=errors,
            compression=compression,
            quoting=quoting,
            cols=columns,
            index_label=index_label,
            mode=mode,
            chunksize=chunksize,
            quotechar=quotechar,
            date_format=date_format,
            doublequote=doublequote,
            escapechar=escapechar,
            storage_options=storage_options,
            formatter=self.fmt,
        )
        csv_formatter.save()

        if created_buffer:
            assert isinstance(path_or_buf, StringIO)
            content = path_or_buf.getvalue()
            path_or_buf.close()
            return content

        return None
#-------------------------------------------------------------------------------------------------------------------------------------------
def prepare_output(
        self,
        path_or_buf: FilePath | WriteBuffer[bytes] | WriteBuffer[str] | None = None,
        sep: str = ",",
        na_rep: str = "",
        float_format: str | None = None,
        columns: Sequence[Hashable] | None = None,
        header: bool_t | list[str] = True,
        index: bool_t = True,
        index_label: IndexLabel | None = None,
        mode: str = "w",
        encoding: str | None = None,
        compression: CompressionOptions = "infer",
        quoting: int | None = None,
        quotechar: str = '"',
        line_terminator: str | None = None,
        chunksize: int | None = None,
        date_format: str | None = None,
        doublequote: bool_t = True,
        escapechar: str | None = None,
        decimal: str = ".",
        errors: str = "strict",
        storage_options: StorageOptions = None,
    ) -> str | None:
    
        df = self if isinstance(self, ABCDataFrame) else self.to_frame()

        formatter = InteloraRecordHandler(
            frame=df,
            header=header,
            index=index,
            na_rep=na_rep,
            float_format=float_format,
            decimal=decimal,
        )

        return InteloraRecordGenerator(formatter).csv_generator(
            path_or_buf,
            line_terminator=line_terminator,
            sep=sep,
            encoding=encoding,
            errors=errors,
            compression=compression,
            quoting=quoting,
            columns=columns,
            index_label=index_label,
            mode=mode,
            chunksize=chunksize,
            quotechar=quotechar,
            date_format=date_format,
            doublequote=doublequote,
            escapechar=escapechar,
            storage_options=storage_options,
        )

In [0]:
# mantain single source data stream and aggregating with table manner
def singleSource(filtered_df):
    #Tone	PositiveTone	NegativeTone	Polarity	ActivityDensity	SelfDensity	WordCount
    filtered_df_result = (filtered_df.groupby(F.date_format("DATE", "y-MM-dd").alias("date"), 'SourceCommonName')
                                      .agg(F.mean("Tone"), F.mean("PositiveTone"), F.mean("NegativeTone"), 
                                           F.mean("Polarity"), F.mean("ActivityDensity"), 
                                           F.mean("SelfDensity"), F.mean("WordCount"))
                                      .withColumn("date", F.to_date("date", format="y-MM-dd"))
                                      .withColumn("date", F.col("date").cast("date"))
                                      .orderBy(F.col("date").asc())
                          )
    filtered_df_result = (filtered_df_result.select("date", "SourceCommonName", F.col("avg(Tone)").alias("Tone"), F.col("avg(PositiveTone)").alias("PositiveTone"), 
                                             F.col("avg(NegativeTone)").alias("NegativeTone"),
                                             F.col("avg(Polarity)").alias("Polarity"), F.col("avg(ActivityDensity)").alias("ActivityDensity"), 
                                             F.col("avg(SelfDensity)").alias("SelfDensity"), F.col("avg(WordCount)").alias("WordCount"))
                                           
                          )
    return filtered_df_result

# calculate average tone data
def avg_day_tone(filtered_df, name):
    """ """
    colname = f"{name.replace(' ', '_')}_tone"
    tone_df = (filtered_df.groupby(date_alignment("DATE", "y-MM-dd").alias("date"))
                          .agg((F.sum("Tone") / F.sum("WordCount")).alias(colname))
                          .select("date", f"{colname}")
                          .withColumn("date", beginning_date("date", format="y-MM-dd"))
                          .withColumn("date", F.col("date").cast("date"))
                          .orderBy(F.col("date").asc())
              )
    return tone_df

# call in make_tables 
# def subtract_cols(df, col1, col2):                                              
#     df = (df.withColumn(col1, df[f"{col1}"] - df[f"{col2}"])
#             .withColumnRenamed(col1, col1.replace("_tone", "_diff")))
#     return df

# getting column filter with date grouping
def get_col_avgs(df):
    exclude = [k for k, v in df.dtypes if v in ["date", "timestamp", "string", "SourceCommonName"]]
    avgs = df.select([F.avg(c).alias(c) for c in df.columns if c not in exclude]).collect()[0]
    return {c: avgs[c] for c in df.columns if c not in exclude}

# # COMMAND ----------

# def show_df_rounded(df, places=4, rows=20):
#     dtypes = {k: v for k, v in df.dtypes}
#     date_cols = [k for k, v in dtypes.items() if v in ["date", "timestamp"]]
#     str_cols = [k for k, v in dtypes.items() if v == "string"]
#     int_cols = [k for k, v in dtypes.items() if "int" in v]
    
#     show_cols = [F.date_format(c, "y-MM-dd").alias(c) if c in date_cols
#                  else (F.col(c).alias(c) if c in str_cols
#                  else (F.format_number(c, 0).alias(c) if c in int_cols
#                  else (F.format_number(c, places).alias(c))))
#                  for c in df.columns]
#     show_cols = [c for c in show_cols]
#     df.select(*show_cols).limit(rows).show()

# # DBTITLE 1,Load Data from Delta Table
# def load_data(save_path, file_name): 
#   df = (spark.read.format("delta")
#                       .option("header", "true")
#                       .option("inferSchema", "true")
#                       .load(os.path.join(save_path, file_name))
#            )
#   return df.toPandas()

# function to filter un wanted data
def filter_non_esg(df): 
    return df[(df['E']==True) | (df['S'] == True) | (df['G'] == True)]

# COMMAND ----------

# web based graph display data generator
class graph_creator:
    def __init__(self, df):
        self.df = df

    def create_graph(self):
        # Find Edges
        df_edge = pd.DataFrame(self.df.groupby("URL").Organization.apply(list)
                               ).reset_index()

        get_tpls = lambda r: (list(itertools.combinations(r, 2)) if
                              len(r) > 1 else None)
        df_edge["SourceDest"] = df_edge.Organization.apply(get_tpls)
        df_edge = df_edge.intelora_data_aligner("SourceDest").to_clean_require(subset=["SourceDest"])

        # Get Weights
        source_dest = pd.DataFrame(df_edge.SourceDest.tolist(),
                                   columns=["Source", "Dest"])
        sd_mapping = source_dest.intelora.arranger(["Source", "Dest"]).size()
        get_weight = lambda r: sd_mapping[r.Source, r.Dest]
        source_dest["weight"] = source_dest.intelora_add(get_weight, axis=1)

        # Get
        self.organizations = set(source_dest.Source.unique()).union(
                             set(source_dest.Dest.unique()))
        self.G = nx.from_pandas_edgelist(source_dest, source="Source",
            target="Dest", edge_attr="weight", create_using=nx.Graph)
        return self.G

# # COMMAND ----------

# function to fit graph with core data
def get_embeddings(G, organizations):
    # Fit graph
    g2v = NVVV()
    g2v.fit(G)
    
    # Embeddings
    embeddings = g2v.model.wv.vectors
    pca = PCA(n_components=3)
    principalComponents = pca.fit_transform(embeddings)
    d_e = pd.DataFrame(principalComponents)
    d_e["company"] = organizations
    return d_e, g2v

# COMMAND ----------

# simple connection bridge function
def get_connections(organizations, topn=25):
    l = [g2v.model.wv.most_similar(org, topn=topn)
         for org in organizations]
    df_sim = pd.DataFrame(l, columns=[f"n{i}" for i in range(topn)])
    for col in df_sim.columns:
        new_cols = [f"{col}_rec", f"{col}_conf"]
        df_sim[new_cols] = pd.DataFrame(df_sim[col].tolist(), 
                                        index=df_sim.index)
    df_sim = df_sim.drop(columns=[f"n{i}" for i in range(topn)])
    df_sim.insert(0, "company", list(organizations))
    return df_sim

# # COMMAND ----------

# connecting with gdelt and then embedating data 
def process_corealgo_and_databrich(start_date,end_date): 
    base_dir = f"dbfs:/mnt/esg/financial_report_data/GDELT_data_Russell_top_300"
    save_dir = os.path.join(base_dir, f"{start}__to__{end}")
    csv_file = "data_as_csv.csv"

    # Load data
    print("Loading Data")
    df = pd.read_csv(os.path.join(save_dir, csv_file).replace("dbfs:/", "/dbfs/"))
    df = filter_non_esg(df)
    # Create graph
    print("Creating Graph")
    creator = graph_creator(df)
    G = creator.create_graph()
    organizations = list(creator.organizations)

    # Save graph as pkl
    fp = os.path.join(save_dir, "organization_graph.pkl").replace("dbfs:/", "/dbfs/")
    with open(fp, "wb") as f:
        pickle.dump(G, f)
        
    # Create embeddings
    print("Creating embeddings")
    emb_path = os.path.join(save_dir, "pca_embeddings.csv").replace("dbfs:/", "/dbfs/")
    d_e, g2v = get_embeddings(G, organizations)
    d_e.to_csv(emb_path, index=False)
    
    # Create connections
    print("Creating connections")
    df_sim = get_connections(organizations)
    sim_path = os.path.join(save_dir, "connections.csv")
    df_sim.to_csv(sim_path.replace("dbfs:/", "/dbfs/"))
    
    # Save organizations as delta
    conn_path = os.path.join(save_dir, "CONNECTIONS")
    conn_data = spark.createDataFrame(df_sim)
    conn_data.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(conn_path)
    
# def make_tables(start_date, end_date):
#      """
#      """
#      # Directories
#     org_types = f"Russell_top_{Fields.n_orgs}"
#     base_dir = f"dbfs:/mnt/esg/financial_report_data/GDELT_data_{org_types}"
#     range_save_dir = os.path.join(base_dir, f"{start_date}__to__{end_date}")
#     esg_dir = os.path.join(range_save_dir, "esg_scores")
#     dbutils.fs.mkdirs(esg_dir)
    
#      # Load data
#     data_path = os.path.join(range_save_dir, "data_as_delta")
#     try:
#         data = (spark.read.format("delta")
#                       .option("header", "true")
#                       .option("inferSchema", "true")
#                       .load(data_path)
#                 )
#          print("Data Loaded!")
#     except:
#          print("Data for these dates hasn't been generated!!!")
#         return

#     # Get all organizations
#     print("Finding all Organizations")
#     organizations = [x.Organization for x in data.select(
#                      "Organization").distinct().collect()]
    
#     # Get the overall tone
#     print("Calculating Tones Over Time")
#     overall_tone = daily_tone(data, "industry")
#     esg_tones = {L: daily_tone(data.filter(f"{L} == True"), "industry")
#                  for L in ["E", "S", "G"]}
    
#     # Loop through the organizations to get the average daily tone for each company
#     pct_idxs = range(0, len(organizations), len(organizations) // 10)
#     for i, org in enumerate(organizations):
#         if i in pct_idxs:
#             print(f"{pct_idxs.index(i) * 10}%")
#         tone_label = f"{org.replace(' ', '_')}_tone"
        
#         overall_org_df = data.filter(f"Organization == '{org}'")
#         org_tone = daily_tone(overall_org_df, org)
#         overall_tone = subtract_cols(overall_tone.join(org_tone, on="date", how="left"),
#                                      tone_label, "industry_tone")
      
#         for L, tdf in esg_tones.items():
#             esg_org_df = overall_org_df.filter(f"{L} == True")
#             esg_org_tone = daily_tone(esg_org_df, org)
#             esg_tones[L] = subtract_cols(tdf.join(esg_org_tone, on="date", how="left"), 
#                                          tone_label, "industry_tone")            
#     del data   
    
#     # Average to get overall scores
#     print("Computing Overall Scores")
#     scores = {}
#     overall_scores = get_col_avgs(overall_tone)
#     esg_scores = {L: get_col_avgs(tdf) for L, tdf in esg_tones.items()}

#     for org in organizations:
#         diff_label = f"{org.replace(' ', '_')}_diff"
#         scores[org] = {L: tdf[diff_label] for L, tdf in esg_scores.items()}
#         scores[org]["T"] = overall_scores[diff_label]
      
      
#     # Save all the tables
#     print("Saving Tables")  
    
#     # Overall ESG
#     print("    Daily Overall ESG")
#     path = os.path.join(esg_dir, "overall_daily_esg_scores.csv").replace("dbfs:/", "/dbfs/")
#     pd_df = overall_tone.toPandas().set_index("date").sort_index().asfreq(freq="D", method="ffill")
#     pd_df.to_csv(path, index=True)
    
#     # E, S, and G
#     for L, tdf in esg_tones.items():
#         print("    Daily " + L)
#         path = os.path.join(esg_dir, f"daily_{L}_score.csv").replace("dbfs:/", "/dbfs/")
#         pd_df = tdf.toPandas().set_index("date").sort_index().asfreq(freq="D", method="ffill")
#         pd_df.to_csv(path, index=True)

#     # Averaged scores
#     print("    Average Scores")
#     score_path = path = os.path.join(esg_dir, "average_esg_scores.csv").replace("dbfs:/", "/dbfs/")
#     pd.DataFrame(scores).to_csv(score_path, index=True)
#     print("DONE!")

In [0]:
def calculating_esg_values(start_date, end_date):
    """
    """
    print("calulation started")
    data_path = f"dbfs:/mnt/esg/financial_report_data/GDELT_data_single_org/data_single_org"
    try:
        data = (spark.read.format("delta")
                     .option("header", "true")
                     .option("inferSchema", "true")
                     .load(data_path)
               )
        print("Data Loaded!")
    except:
        print("Data for these dates hasn't been generated!!!")
        return

    # Get all organizations
    print("Finding all Organizations")
    organizations = [x.Organization for x in data.select(
                     "Organization").distinct().collect()]
    
    # Get the overall tone
    print(organizations)
    print("Calculating Tones Over Time")
    overall_tone = avg_day_tone(singleSource(data), "industry")                                              
    esg_tones = {L: avg_day_tone(singleSource(data.filter(f"{L} == True")), "industry")
                 for L in ["E", "S", "G"]}
    
    pct_idxs = range(0, len(organizations))
    
    for i, org in enumerate(organizations):
        tone_label = f"{org.replace(' ', '_')}_tone"

        overall_org_df = data.filter(f"Organization == '{org}'")
        org_tone = avg_day_tone(singleSource(overall_org_df), org)

        overall_tone = overall_tone.join(org_tone, on="date", how="left")
      
        for L, tdf in esg_tones.items():
            esg_org_df = overall_org_df.filter(f"{L} == True")
            esg_org_tone = avg_day_tone(singleSource(esg_org_df), org)
            
            esg_tones[L] = tdf.join(esg_org_tone, on="date", how="left")
    
    del data 
    
    # get tone data for pre processing
    shouls_keep_garbaze = "True"
    if shouls_keep_garbaze == "false": 
        make_embeddings_and_connections(start_date,end_date)
    else:
        # Average to get overall scores
        print(organizations)
        print("Computing Overall Scores")
        scores = {}
        print("    Calculating overall tone")
        overall_scores = get_col_avgs(overall_tone)
        print("    Calculating esg tone")
        esg_scores = {L: get_col_avgs(tdf) for L, tdf in esg_tones.items()}
    
        #print(overall_scores)
        print(esg_scores)
        print("DONE!")
        return esg_scores, overall_tone
        

In [0]:
# Begin process to download data for calculation
def download_company_data(base_dir, start_date, end_date, org_name):
    #print(org_name)
    _ = download_gdelt_data(start_date, end_date,org_name, save_csv=True)
    #print(org_name)
    esg_score1, _ = calculating_esg_values(start_date, end_date)
    return esg_score1

### Calculating ESG Scores for input Companies:

In [0]:
# web REST module to take input
base_dir = "dbfs:/mnt/esg/financial_report_data" 
dbutils.widgets.text("myinput","microsoft;apple")
dbutils.widgets.text("startdate","2022-05-01")
dbutils.widgets.text("enddate","2022-05-02")

var_a = dbutils.widgets.get("myinput")
var_a = var_a.split(';')
print(var_a)
start_date = dbutils.widgets.get("startdate")
end_date = dbutils.widgets.get("enddate")

['microsoft', 'apple']

In [0]:
# manage company data during date period
output = download_company_data(base_dir, start_date, end_date, var_a)
dbutils.notebook.exit(output)


{'E': {'industry_tone': -0.00037340860057307433, 'microsoft_tone': -0.00042677988406399217, 'apple_tone': 0.002173904392291133}, 'S': {'industry_tone': 0.0001253949109221482, 'microsoft_tone': 0.0004051838667690962, 'apple_tone': -0.0009927997403526272}, 'G': {'industry_tone': -0.0006220543071157147, 'microsoft_tone': -0.0007719906056858187, 'apple_tone': -2.187044036061831e-05}}